### NLP

In [16]:
import pandas as pd
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import string

stop_words = set(stopwords.words('german'))

[nltk_data] Downloading package stopwords to /Users/clara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/clara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/clara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Extract the most important words from title, h1, abstract, url with NLP (e.g. TF-IDF )

In [38]:
df_nlp = pd.read_csv('/Users/clara/Desktop/neuefische/d-drivers/data/data_nlp.csv')

In [39]:
df_nlp.columns

Index(['page_id', 'n_days', 'url', 'n_urls', 'no_versions', 'age',
       'last_publish_date', 'publish_date_min', 'word_count',
       'classification_product', 'classification_type', 'page_name', 'title',
       'last_author', 'external_clicks', 'external_impressions',
       'likes_n_days', 'dislikes_n_days', 'video_play', 'page_impressions',
       'clickouts', 'ctr', 'h1', 'scraped_author', 'date_scraped', 'abstract',
       'scraped_word_count', 'meta_title', 'meta_description',
       'meta_image_url', 'media_type', 'page_img_size', 'merged_url',
       'meta_title_len', 'meta_desc_len', 'h1_len', 'abstract_len',
       'merged_url_len', 'sentiment_abstract', 'confidence_abstract',
       'sentiment_meta_title', 'confidence_meta_title'],
      dtype='object')

In [40]:
# df_nlp = df[['page_id', 'external_clicks', 'external_impressions', 'ctr', 'h1', 
#        'abstract', 'meta_title', 'meta_description', 'merged_url']]

In [41]:
df_nlp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6815 entries, 0 to 6814
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   page_id                 6815 non-null   int64  
 1   n_days                  6815 non-null   int64  
 2   url                     6815 non-null   object 
 3   n_urls                  6815 non-null   int64  
 4   no_versions             6815 non-null   int64  
 5   age                     6815 non-null   int64  
 6   last_publish_date       6815 non-null   object 
 7   publish_date_min        6815 non-null   object 
 8   word_count              6815 non-null   float64
 9   classification_product  6815 non-null   object 
 10  classification_type     6815 non-null   object 
 11  page_name               6815 non-null   object 
 12  title                   6815 non-null   object 
 13  last_author             6815 non-null   object 
 14  external_clicks         6815 non-null   

In [42]:
df_nlp.head()

,page_id,n_days,url,n_urls,no_versions,age,last_publish_date,publish_date_min,word_count,classification_product,...,merged_url,meta_title_len,meta_desc_len,h1_len,abstract_len,merged_url_len,sentiment_abstract,confidence_abstract,sentiment_meta_title,confidence_meta_title
0,1037,6,https://efahrer.chip.de/e-wissen/elektrofahrze...,2,0,22,2024-03-10,2024-03-10,827.0,E-Auto,...,"['elektrofahrzeug', 'ladestation']",55,64,61,217.0,2,positive,0.891435,neutral,0.994909
1,1039,1,https://efahrer.chip.de/e-wissen/elektroauto-f...,2,0,697,2022-05-05,2022-05-05,1066.0,E-Auto,...,"['absahnen', 'koennen', 'satt', '2024', 'sie',...",46,141,75,486.0,11,neutral,0.950322,neutral,0.940748
2,1040,10,https://efahrer.chip.de/news/haetten-sie-es-ge...,1,0,40,2024-02-21,2024-02-21,466.0,E-Auto,...,"['so', 'gewusst', 'e', 'haetten', 'autos', 'si...",53,155,53,264.0,10,negative,0.980951,neutral,0.967907
3,10245,3,https://efahrer.chip.de/news/kleinste-auto-der...,1,0,2282,2018-01-01,2018-01-01,0.0,Auto,...,"['zurueck', 'welt', 'fans', 'koennen', 'kommt'...",71,156,71,314.0,12,neutral,0.999933,neutral,0.962445
4,10273,24,https://efahrer.chip.de/news/winter-tipps-fuer...,4,4,2282,2024-02-28,2018-01-01,530.5,E-Auto,...,"['so', 'e', 'ihrem', 'raus', 'reichweite', 'au...",72,152,72,262.0,14,neutral,0.899187,negative,0.946896


### Scale target variable

In [43]:
scaler = StandardScaler()
scaler.fit(df_nlp[['external_impressions']])
# Transform the target variable
df_nlp['external_impressions_scaled'] = scaler.transform(df_nlp[['external_impressions']])

### Remove stop words

In [44]:
def remove_stopwords(text):
    if isinstance(text, str):        
        words = word_tokenize(text)
        # Remove punctuation and special characters
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove stopwords
        return ' '.join([word for word in words if word.lower() not in stop_words])
    else:
        return text

def remove_stopwords_from_columns(df, columns):
    for col in columns:
        df[col] = df[col].apply(remove_stopwords)
    return df

columns_to_clean = ['h1','abstract','meta_title','meta_description','merged_url']
df_nlp = remove_stopwords_from_columns(df_nlp, columns_to_clean)

In [45]:
df_nlp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6815 entries, 0 to 6814
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   page_id                      6815 non-null   int64  
 1   n_days                       6815 non-null   int64  
 2   url                          6815 non-null   object 
 3   n_urls                       6815 non-null   int64  
 4   no_versions                  6815 non-null   int64  
 5   age                          6815 non-null   int64  
 6   last_publish_date            6815 non-null   object 
 7   publish_date_min             6815 non-null   object 
 8   word_count                   6815 non-null   float64
 9   classification_product       6815 non-null   object 
 10  classification_type          6815 non-null   object 
 11  page_name                    6815 non-null   object 
 12  title                        6815 non-null   object 
 13  last_author       

In [46]:
df_nlp.to_csv('/Users/clara/Desktop/neuefische/d-drivers/data/preprocessing_nlp.csv', encoding='utf-8', index=False)


### Use pretrained model

In [11]:
from transformers import pipeline
import pandas as pd
import numpy as np
import tensorflow
import torch
from tqdm import tqdm
import sentencepiece

In [12]:
data = df_nlp

print(f'There are {data.shape[0]} rows in the dataset')

There are 6815 rows in the dataset


In [13]:
#Preparing the pipeline in one-line of code!
#classifier = pipeline("zero-shot-classification",device = 0)
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# Define the function get_predictions_score
def get_predictions_score(prediction):
    pred_labels = prediction['labels']
    pred_scores = prediction['scores']
    
    # Find the index of the label with the highest probability
    max_index = pred_scores.index(max(pred_scores))
    
    # Extract the label and its corresponding probability
    max_label = pred_labels[max_index]
    max_probability = pred_scores[max_index]
    
    return max_label, max_probability

# Define the sequence, candidate_labels, and make predictions
sequence = df_nlp['h1'].iloc[1]  # Assuming 'h1' is the column you want to process
candidate_labels = ['Elektroauto-Bewertungen', 'Ladeinfrastruktur für Elektroautos', 'Technologien für Elektroautos', 'Batterietechnologie für Elektroautos', 'Trends auf dem Markt für Elektrofahrzeuge', 'Neuigkeiten von Elektroautoherstellern', 'Tipps zur Wartung von Elektroautos', 'Vorschriften in der Elektrofahrzeugindustrie', 'Nachhaltigkeit und Umweltauswirkungen von Elektroautos', 'Vergleich von Elektrofahrzeugen', 'Reichweite und Leistung von Elektroautos', 'Sicherheitsmerkmale von Elektroautos', 'Steueranreize und Rabatte für Elektroautos', 'Kosten für den Besitz eines Elektroautos', 'Tipps und Strategien zum Laden von Elektroautos', 'Veranstaltungen und Konferenzen zum Thema Elektrofahrzeuge', 'Erfahrungen und Testimonials von Elektroautobesitzern', 'Modelle und Spezifikationen von Elektroautos', 'Software-Updates und Funktionen von Elektrofahrzeugen', 'Innovationen und zukünftige Trends im Bereich Elektroautos']
pred = classifier(sequence, candidate_labels)

# Apply the function to the DataFrame column and save the result in new columns
df_nlp['predicted_label'], df_nlp['predicted_probability'] = zip(*df_nlp['h1'].apply(lambda x: get_predictions_score(classifier(x, candidate_labels))))

# Display the DataFrame with the new columns
print(df_nlp)

      page_id  external_clicks  external_impressions       ctr   
0        1037            256.0                5444.0  4.702425  \
1        1039            124.0                1728.0  7.175926   
2        1040             83.0                2575.0  3.223301   
3       10245             80.0                 845.0  9.467456   
4       10273           3384.0               94132.0  3.594952   
...       ...              ...                   ...       ...   
6810  1018768         189254.0             2254612.0  8.394083   
6811  1018770           3822.0               48838.0  7.825873   
6812  1018771           1492.0               32728.0  4.558788   
6813  1018776            347.0                6890.0  5.036284   
6814  1018782           1202.0               29790.0  4.034911   

                                                     h1   
0     Elektrofahrzeug-Ladestation : Kosten Anbieter ...  \
1          Elektroauto-Förderung 2024 : satt absahnen -   
2             Hätten gewusst ?

/var/folders/5s/w7hst37d0tv0r4hby2_mqqp00000gn/T/ipykernel_14333/490298415.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['predicted_label'], df_nlp['predicted_probability'] = zip(*df_nlp['h1'].apply(lambda x: get_predictions_score(classifier(x, candidate_labels))))
/var/folders/5s/w7hst37d0tv0r4hby2_mqqp00000gn/T/ipykernel_14333/490298415.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['predicted_label'], df_nlp['predicted_probability'] = zip(*df_nlp['h1'].apply(lambda x: get_predi

In [17]:
df_nlp.head()

,page_id,external_clicks,external_impressions,ctr,h1,abstract,meta_title,meta_description,merged_url,predicted_label,predicted_probability
0,1037,256.0,5444.0,4.702425,Elektrofahrzeug-Ladestation : Kosten Anbieter ...,"Grunde E-Auto überall laden , Strom gibt . Tat...","Elektrofahrzeug-Ladestation : Anbieter , Koste...",Ladestationen Elektroautos . Kosten Anbieter V...,"[ 'elektrofahrzeug ' , 'ladestation ' ]",Ladeinfrastruktur für Elektroautos,0.431657
1,1039,124.0,1728.0,7.175926,Elektroauto-Förderung 2024 : satt absahnen -,große staatliche Förderung Elektroautos beende...,E-Auto Förderung 2024 : Infos Überblick,E-Auto Prämie 2024- beim Kauf Geld sparen . Fö...,"[ 'elektroauto ' , 'und ' , '2024 ' , 'koennen...",Vorschriften in der Elektrofahrzeugindustrie,0.303427
2,1040,83.0,2575.0,3.223301,Hätten gewusst ? lange gibt schon E-Autos,wenigen Jahren wurden Elektroautos absolute Ne...,Hätten gewusst ? lange gibt schon E-Autos,wenigen Jahren wurden Elektroautos absolute Ne...,"[ 'schon ' , 'so ' , 'lange ' , 'autos ' , 'ha...",Technologien für Elektroautos,0.296509
3,10245,80.0,845.0,9.467456,Kleinstes Auto Welt kommt zurück : Fans selber...,"P.50 , bekannt kleinste Auto Welt , gilt Samml...",Kleinstes Auto Welt kommt zurück : Fans selber...,"P.50 , bekannt kleinste Auto Welt , gilt Samml...","[ 'auto ' , 'koennen ' , 'bauen ' , 'jetzt ' ,...",Elektroauto-Bewertungen,0.128451
4,10273,3384.0,94132.0,3.594952,Mehr Reichweite Winter : holen E-Auto-Akku raus,Wer freut schon eisige Kälte ? E-Autos sicherl...,Mehr Reichweite Winter : holen E-Auto-Akku raus,Wer freut schon eisige Kälte ? E-Autos sicherl...,"[ 'mehr ' , 'auto ' , 'holen ' , 'so ' , 'wint...",Batterietechnologie für Elektroautos,0.295967


In [22]:
df_nlp.to_csv('/Users/clara/Desktop/neuefische/d-drivers/data/df_nlp_huggingface.csv', encoding='utf-8', index=False)

### Vertex 

### Stemming

In [ ]:
# tbd

### Vectorizing

In [ ]:
df_nlp_vec = df_nlp.copy()
df_nlp_vec.fillna('', inplace=True)
df_nlp_vec.isna().sum() #info()

In [ ]:
def vectorize_text(column, df):
    col = df[column]
    vect = CountVectorizer().fit(col)
    transformed = vect.transform(col)
    for i, feature_name in enumerate(vect.get_feature_names_out()):
        df[f'{column}_{feature_name}'] = transformed[:, i].toarray().flatten()  # Convert to dense array
    return df

for item in ['meta_description']:
    df = vectorize_text(column=item, df=df_nlp_vec)
    filename = f'../data/nlp_features_{item}.csv'
    df.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
def vectorize_text(column, df):
    col = df[column]
    vect = CountVectorizer().fit(col)
    transformed = vect.transform(col)
    for i, feature_name in enumerate(vect.get_feature_names_out()):
        df[f'{column}_{feature_name}'] = transformed[:, i].toarray().flatten()  # Convert to dense array
    return df

for item in ['merged_url']:
    df = vectorize_text(column=item, df=df_nlp_vec)
    filename = f'../data/nlp_features_{item}.csv'
    df.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
def vectorize_text(column, df):
    col = df[column]
    vect = CountVectorizer().fit(col)
    transformed = vect.transform(col)
    
    # Create a DataFrame from the transformed array
    df_transformed = pd.DataFrame(transformed.toarray(), columns=vect.get_feature_names_out(), index=df.index)
    
    # Concatenate the new DataFrame with the original DataFrame
    df_concatenated = pd.concat([df, df_transformed], axis=1)
    
    # Drop the original column
    df_concatenated.drop(columns=[column], inplace=True)
    
    return df_concatenated

# Iterate over columns to clean
for item in columns_to_clean:
    df_nlp_vec = vectorize_text(column=item, df=df_nlp_vec)
    filename = f'../data/nlp_features_{item}.csv'
    df_nlp_vec.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
df_full_vec = pd.merge('nlp_features_h1','nlp_features_abstract',how='left',left_index=True)

In [ ]:
df_full_vec = pd.merge(nlp_features_h1, nlp_features_abstract, how='left', left_index=True, right_index=True)

In [13]:
from transformers import pipeline
import pandas as pd
import numpy as np
from tqdm import tqdm
data = pd.read_csv(
    "data/SMSSpamCollection.txt",
    encoding="utf-8",
    header=None,
    delimiter="\t",
    names=["target", "text"],
)
data.head(5)
print(f'There are {data.shape[0]} rows in the dataset')
Preparing the pipeline in one-line of code!
classifier = pipeline("zero-shot-classification",device = 0)

SyntaxError: invalid syntax (3410687870.py, line 14)